<a href="https://colab.research.google.com/github/kitsamho/imdb_app/blob/main/Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.0.2


In [1]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics

import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)


2021-08-04 16:34:41.006913: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
!pip install simpletransformers

  Using cached transformers-4.9.1-py3-none-any.whl (2.6 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Read in Move Data
---

SyntaxError: ignored

In [3]:
df = pd.read_json('/content/drive/MyDrive/tmdb_data_main.json')

In [4]:
df = df[['tmdb_id','movie','overview','genres','genres_new','popularity','release_year','vote_average','budget','revenue','poster_path']]
df = df.dropna(subset=['genres'])
genre_include = df.explode('genres').genres.value_counts().keys()[0:10]

In [5]:


def genres_check(film_genres):
    
    for genre in film_genres:
        if genre not in genre_include:
            return False
    return True

def multi_label_prep(film_genres):
    prep = []
    for genre in genre_include:
        if genre in film_genres:
            prep.append(1)
        else:
            prep.append(0)
            
    return prep

df['genres_valid'] = df['genres'].apply(genres_check)
df = df[df.genres_valid ==True]
df.shape
genre_include = list(genre_include)
genre_include.sort()
genre_include



df['labels'] = df['genres'].apply(multi_label_prep)

In [6]:
df = pd.concat([df[['overview']],df[['labels']]['labels'].apply(pd.Series)],axis=1)

In [15]:
df.columns = ['overview']+genre_include
df = df.reset_index(drop=True)

In [22]:
res = []
for i in range(df.shape[0]):
  res.append(list(df[df.columns[1:]].loc[i].values))

In [28]:
df['labels'] = pd.DataFrame(pd.Series(res))

In [30]:
df['text'] = df['overview']

In [34]:
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(df, test_size=0.2)

In [9]:
import simpletransformers


In [31]:
import torch
cuda_available = torch.cuda.is_available()

In [26]:
import transformers

In [11]:
from simpletransformers.classification import MultiLabelClassificationModel

In [32]:
print(cuda_available)

True


In [33]:
model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=10, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 5, 'max_seq_length': 512},use_cuda=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bia

In [36]:
model.train_model(train_df[['labels','text']])

  0%|          | 0/9848 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/4924 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/4924 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/4924 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/4924 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/4924 [00:00<?, ?it/s]

(1535, 0.27520917841075565)

In [37]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/2463 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/308 [00:00<?, ?it/s]

In [42]:
to_predict = ["Oskar Schindler, a German industrialist and member of the Nazi party, tries to save his Jewish employees after witnessing the persecution of Jews in Poland."]
preds, outputs = model.predict(to_predict)

sub_df = pd.DataFrame(outputs, columns=genre_include)
sub_df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,Action,Adventure,Comedy,Crime,Drama,Family,Horror,Romance,Science Fiction,Thriller
0,0.030792,0.010857,0.032593,0.162476,0.978027,0.007038,0.007576,0.044006,0.002878,0.21521


# New Section